# LangChain 实战：洗衣机销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chatgpt.com/share/b12a3a92-04ef-4b42-a5c5-b862aaaaca16


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的洗衣机销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
1.
[客户问题]：这款洗衣机的洗涤效果怎么样？
[销售回答]：这款洗衣机采用了先进的洗涤技术，能深层清洁衣物，即使是顽固污渍也能轻松去除，保证您的衣物洁净如新。

2.
[客户问题]：这款洗衣机省电吗？
[销售回答]：是的，这款洗衣机采用了节能设计，能耗低，长期使用可以大幅减少电费支出，是非常经济实惠的选择。

3.
[客户问题]：我家空间比较小，这款洗衣机放得下吗？
[销售回答]：这款洗衣机的设计非常紧凑，占地面积小，即使在小空间内也能轻松放置，完全适合您的需求。

4.
[客户问题]：这款洗衣机噪音大吗？
[销售回答]：这款洗衣机配备了静音技术，运行时非常安静，几乎不会打扰到您和家人的日常生活。

5.
[客户问题]：这款洗衣机使用起来方便吗？
[销售回答]：非常方便，这款洗衣机具有智能操作面板，功能直观易懂，任何人都能轻松上手。

6.
[客户问题]：洗衣机容量大吗？
[销售回答]：这款洗衣机有多种容量可供选择，您可以根据家庭人口和洗涤需求选择合适的型号，确保能一次性洗涤更多衣物。

7.
[客户问题]：这款洗衣机对衣物的磨损大吗？
[销售回答]：这款洗衣机采用了柔性洗涤技术，能够在清洁衣物的同时，最大程度减少衣物的磨损，保护您的衣物质地。

8.
[客户问题]：这款洗衣机怎么控制水温？
[销售回答]：这款洗衣机配备了智能水温控制系统，您可以根据不同的衣物材质，选择适合的水温进行洗涤，非常方便。

9.
[客户问题]：这款洗衣机有什么特殊功能吗？
[销售回答]：是的，这款洗衣机除了常规的洗涤功能外，还配备了快速洗、消毒、除菌等多种特殊功能，满足您各种洗衣需求。

10.
[客户问题]：售后服务怎么样？
[销售回答]：我们的售后服务非常完善，提供免费安装、保修等服务，还有24小时服务热线，随时为您解决问题，让您买得安心，用得放心。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 wash_machine_sales_data.txt 文件中

In [1]:
with open("wash_machine_sales_data.txt") as f:
    wash_machine_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([wash_machine_sales])

In [5]:
docs[0]

Document(page_content='[客户问题]：这款洗衣机的洗涤效果怎么样？\n[销售回答]：这款洗衣机采用了先进的洗涤技术，能深层清洁衣物，即使是顽固污渍也能轻松去除，保证您的衣物洁净如新。')

In [6]:
len(docs)

91

In [7]:
#环境变量配置，不需要可以删除
import os
os.environ['OPENAI_API_KEY'] = 'sk-9GLvQswLxSJj1Uzq6d1fFc7cDb614eA4A7C60bCf2cA77483'
os.environ['OPENAI_BASE_URL']='https://apikeyplus.com/v1'

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [9]:
query = "洗衣机噪音大吗"

In [10]:
answer_list = db.similarity_search(query)

In [11]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]：这款洗衣机噪音大吗？
[销售回答]：这款洗衣机配备了静音技术，运行时非常安静，几乎不会打扰到您和家人的日常生活。

[客户问题]：这款洗衣机会不会产生噪音？
[销售回答]：这款洗衣机采用了静音技术，运行时噪音极低，不会打扰到您的日常生活。

[客户问题]：洗衣机会不会影响到楼下的邻居？
[销售回答]：不会的，这款洗衣机采用了高效减震技术，运行时振动小，即使在夜间使用，也不会影响到楼下的邻居。

[客户问题]：洗衣机会不会产生振动？
[销售回答]：不会的，这款洗衣机配备了平衡系统，可以有效减少洗衣机在高速运转时产生的振动，保持稳定安静的运行。



In [12]:
db.save_local("wash_machine_sales")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [13]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [14]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x13694dcf0>, search_kwargs={'k': 3})

In [15]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题]：这款洗衣机噪音大吗？
[销售回答]：这款洗衣机配备了静音技术，运行时非常安静，几乎不会打扰到您和家人的日常生活。

[客户问题]：这款洗衣机会不会产生噪音？
[销售回答]：这款洗衣机采用了静音技术，运行时噪音极低，不会打扰到您的日常生活。

[客户问题]：洗衣机会不会影响到楼下的邻居？
[销售回答]：不会的，这款洗衣机采用了高效减震技术，运行时振动小，即使在夜间使用，也不会影响到楼下的邻居。



In [16]:
docs = topK_retriever.get_relevant_documents("你们有没有8公斤的洗衣机啊？")

In [17]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：这款洗衣机的容量是多少？
[销售回答]：这款洗衣机有8公斤、10公斤和12公斤三种容量可供选择，您可以根据家庭的需求选择适合的型号。

[客户问题]：洗衣机容量大吗？
[销售回答]：这款洗衣机有多种容量可供选择，您可以根据家庭人口和洗涤需求选择合适的型号，确保能一次性洗涤更多衣物。

[客户问题]：这款洗衣机能不能清洁大件物品？
[销售回答]：当然可以，这款洗衣机容量大且动力强劲，像被子、窗帘等大件物品也能轻松清洗，完全不用担心。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [18]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [19]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：这款洗衣机噪音大吗？
[销售回答]：这款洗衣机配备了静音技术，运行时非常安静，几乎不会打扰到您和家人的日常生活。

[客户问题]：这款洗衣机会不会产生噪音？
[销售回答]：这款洗衣机采用了静音技术，运行时噪音极低，不会打扰到您的日常生活。

[客户问题]：洗衣机会不会影响到楼下的邻居？
[销售回答]：不会的，这款洗衣机采用了高效减震技术，运行时振动小，即使在夜间使用，也不会影响到楼下的邻居。



### 提取向量数据库中的`销售回答`

In [20]:
docs = retriever.get_relevant_documents(query)

In [21]:
docs[0].page_content

'[客户问题]：这款洗衣机噪音大吗？\n[销售回答]：这款洗衣机配备了静音技术，运行时非常安静，几乎不会打扰到您和家人的日常生活。'

In [22]:
docs[0].page_content.split("[销售回答] ")

['[客户问题]：这款洗衣机噪音大吗？\n[销售回答]：这款洗衣机配备了静音技术，运行时非常安静，几乎不会打扰到您和家人的日常生活。']

In [23]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [24]:
ans

'[客户问题]：这款洗衣机噪音大吗？\n[销售回答]：这款洗衣机配备了静音技术，运行时非常安静，几乎不会打扰到您和家人的日常生活。'

#### 尝试各种问题

In [25]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [26]:
query = "我想买一个顶配的洗衣机"

print(sales(query))

[]


/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [27]:
print(sales(query, 0.75))

['[客户问题]：洗衣机容量大吗？\n[销售回答]：这款洗衣机有多种容量可供选择，您可以根据家庭人口和洗涤需求选择合适的型号，确保能一次性洗涤更多衣物。', '[客户问题]：这款洗衣机使用起来方便吗？\n[销售回答]：非常方便，这款洗衣机具有智能操作面板，功能直观易懂，任何人都能轻松上手。', '[客户问题]：这款洗衣机的容量是多少？\n[销售回答]：这款洗衣机有8公斤、10公斤和12公斤三种容量可供选择，您可以根据家庭的需求选择适合的型号。', '[客户问题]：这款洗衣机适合小户型使用吗？\n[销售回答]：非常适合，这款洗衣机设计紧凑，占地面积小，即使在小户型中也能轻松摆放。']


In [28]:
query = "价格2000元以上"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['[客户问题]：这个价格有优惠吗？\n[销售回答]：我们目前正在做促销活动，您购买这款洗衣机还可以享受折扣优惠，性价比非常高，建议您尽快下单哦。', '[客户问题]：这款洗衣机是进口的吗？\n[销售回答]：这款洗衣机采用了国际先进的技术和进口的核心部件，质量有保证，性能卓越。', '[客户问题]：售后服务怎么样？\n[销售回答]：我们的售后服务非常完善，提供免费安装、保修等服务，还有24小时服务热线，随时为您解决问题，让您买得安心，用得放心。', '[客户问题]：这款洗衣机的电机是多大功率的？\n[销售回答]：这款洗衣机配备了高效节能电机，功率为2000瓦，既能保证强劲的洗涤效果，又非常节能省电。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [30]:
qa_chain({"query": "你们有2000元以上的洗衣机吗？"})

/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '你们有2000元以上的洗衣机吗？',
 'result': '[销售回答]：当然有，我们店里有多种品牌和型号的洗衣机，价格区间覆盖了从经济型到高端型，2000元以上的洗衣机选择也很丰富。您可以根据您的具体需求和偏好来选择合适的洗衣机。'}

In [31]:
qa_chain({"query": "洗衣机吵不吵"})

{'query': '洗衣机吵不吵', 'result': '这款洗衣机配备了静音技术，运行时噪音极低，因此不会很吵，几乎不会打扰到您和家人的日常生活。'}

In [32]:
print(sales("洗衣机吵不吵"))

['[客户问题]：洗衣机会不会影响到楼下的邻居？\n[销售回答]：不会的，这款洗衣机采用了高效减震技术，运行时振动小，即使在夜间使用，也不会影响到楼下的邻居。', '[客户问题]：这款洗衣机噪音大吗？\n[销售回答]：这款洗衣机配备了静音技术，运行时非常安静，几乎不会打扰到您和家人的日常生活。', '[客户问题]：这款洗衣机会不会产生噪音？\n[销售回答]：这款洗衣机采用了静音技术，运行时噪音极低，不会打扰到您的日常生活。', '[客户问题]：洗衣机的操作难吗？\n[销售回答]：一点也不难，这款洗衣机设计了智能操作界面，简单易用，即使是老人或小孩也能轻松操作。']


## 加载 FAISS 向量数据库已有结果

In [33]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 不添加 allow_dangerous_deserialization=True 会报错，参考：https://stackoverflow.com/questions/78120202/the-de-serialization-relies-loading-a-pickle-file
db = FAISS.load_local("wash_machine_sales", OpenAIEmbeddings(),allow_dangerous_deserialization=True)

In [34]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [35]:
qa_chain({"query": "我想买带烘干的洗衣机，你们有么"})

{'query': '我想买带烘干的洗衣机，你们有么', 'result': '是的，我们有带独立烘干模式的洗衣机，非常方便实用。'}

In [36]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [37]:
qa_chain({"query": "我想买带烘干的洗衣机，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买带烘干的洗衣机，你们有么', 'result': '是的，我们有带独立烘干模式的洗衣机，非常方便实用。'}

In [38]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [39]:
result = qa_chain({"query": "我想买带烘干的洗衣机，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [40]:
result

{'query': '我想买带烘干的洗衣机，你们有么',
 'result': '是的，我们有带独立烘干模式的洗衣机。',
 'source_documents': [Document(page_content='[客户问题]：这款洗衣机有单独的烘干模式吗？\n[销售回答]：是的，这款洗衣机配有独立的烘干模式，您可以选择只烘干衣物，非常方便实用。')]}